## 04) 연관어분석(Network Analysis)

- **My project**
    - 방식: 1.동시출현 빈도기반, 2.통계적유사도, 3.딥러닝Word2Vec 방법 중 동시출현빈도기반 방식 사용
        1. customized 함수 만들어서 문서를 한 문맥으로+문서 내 중복단어 한개로보는 방법 (사용 O)
        2. nltk.collocation함수 window지정하는방법 (사용 X)
    - 주제
        1. 불가리 전체 nouns에서 네트워크
        2. 특정 키워드(예:목걸이)에서 네트워크

In [1]:
import pickle
import pandas as pd
import numpy as np
import nltk

import warnings # 경고 알림 제거
warnings.filterwarnings("ignore", category=Warning) # 경고 알림이 뜨면 모두 무시합니다.

In [2]:
# 시각화
from matplotlib import pyplot as plt
# import seaborn as sns
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False # 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
# plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['font.family'] = 'Gulim'

In [3]:
pd.set_option('display.float_format', lambda x: '%.3f' % x) #.0f: 소수점 자리수 지정
# pd.set_option('display.min_rows', 20)
# pd.set_option('display.max_colwidth', None) #default 50

#### 1. 동시출현 빈도기반 함수

In [4]:
input_file_name = "data/BCTV_v2.pk"
with open(input_file_name, 'rb') as f: # read as binary(피클 파일 읽기 위하여)
    data = pickle.load(f)

In [5]:
len(data)

17115

In [6]:
df = data.drop_duplicates(subset=['article_titles', 'nick_names', 'dates']) #category_id로 제거하는것보다 더 제거됨(똑같은 게시글을 다른 게시판에 복붙?)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14080 entries, 0 to 17114
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   category_id      14080 non-null  object        
 1   article_type     14080 non-null  object        
 2   article_titles   14080 non-null  object        
 3   nick_names       14080 non-null  object        
 4   dates            14080 non-null  object        
 5   num_views        14080 non-null  int32         
 6   article_content  14080 non-null  object        
 7   num_comment      14080 non-null  int64         
 8   comment_list     14080 non-null  object        
 9   new_dates        14080 non-null  datetime64[ns]
 10  year             14080 non-null  int64         
 11  month            14080 non-null  int64         
 12  day              14080 non-null  int64         
 13  weekday          14080 non-null  int64         
 14  nouns_title      14080 non-null  objec

In [10]:
%%time
stop_words = ['해주시']
count = {}   #동시출현 빈도가 저장될 dict
for tokens in df['nouns_document']: #동시출현빈도 세기위한 문맥은 문서단위(개별 리뷰한개를 문서 한개로 봄) - 일반적
    stopped_tokens = [i for i in list(set(tokens)) if not i in stop_words] #동일한 문맥 내에서 중복 발생한 단어는 모두 하나로 봄
    for i, a in enumerate(stopped_tokens):
        for b in stopped_tokens[i+1:]:
            if a>b: #알파벳 순서인듯 : a가 더 알파벳순 뒤에
                count[b, a] = count.get((b, a),0) + 1  # get(x) 함수는 x라는 Key에 대응되는 Value, 딕셔너리 안에 찾으려는 Key 값이 없을 경우 미리 정해 둔 디폴트 값을 대신 가져오게 하고 싶을 때에는 get(x, '디폴트 값') 사용하면 편리
            else :
                count[a, b] = count.get((a, b),0) + 1     

# dictionary를 바로 df으로 만드는 코드
df1=pd.DataFrame.from_dict(count, orient='index')

Wall time: 40.2 s


#### 2. Gephi를 이용해 그래프를 그리기 위해서 두 개의 테이블 형태의 데이터를 추출

##### 2-1. edge table

In [11]:
%%time
# bigram index를 두개의 행으로 분리
list1=[]
for i in range(len(df1)):
    list1.append([df1.index[i][0],df1.index[i][1],df1[0][i]])

df_key=pd.DataFrame(list1, columns = ['Source','Target','Weight']).sort_values('Weight', ascending=False, ignore_index=True)

Wall time: 57.1 s


In [12]:
df_key.loc[:10]

,Source,Target,Weight
0,까르띠에,시계,3175
1,목걸이,반클리프,2523
2,까르띠에,매장,2293
3,구매,까르띠에,2291
4,까르띠에,추천,2090
5,까르띠에,팔찌,1956
6,가격,까르띠에,1946
7,구매,매장,1879
8,까르띠에,착용,1849
9,까르띠에,다이아,1833


In [13]:
# 특정 키워드 연관어 추출
df_key = df_key[(df_key['Source']=='부쉐론')|(df_key['Target']=='부쉐론')|(df_key['Source']=='쎄뻥')|(df_key['Target']=='쎄뻥')|(df_key['Source']=='콰트로')|(df_key['Target']=='콰트로')]
df_key = df_key.reset_index(drop=True)

In [32]:
print(df_key.describe())

threshold_value = 0.995
print(df_key['Weight'].quantile(q=threshold_value, interpolation='nearest'))
threshold = df_key['Weight'].quantile(q=threshold_value, interpolation='nearest')

         Weight
count 13731.000
mean      4.508
std      12.804
min       1.000
25%       1.000
50%       1.000
75%       3.000
max     319.000
77


In [33]:
edge_df = df_key[df_key['Weight'] >= threshold]
len(edge_df)

71

In [34]:
edge_df[(edge_df['Source']=='까르띠에')|(edge_df['Source']=='반클리프')|(edge_df['Source']=='불가리')|(edge_df['Source']=='티파니')\
       |(edge_df['Target']=='까르띠에')|(edge_df['Target']=='반클리프')|(edge_df['Source']=='불가리')|(edge_df['Target']=='티파니')]

,Source,Target,Weight
3,반클리프,부쉐론,284
5,까르띠에,부쉐론,268
8,부쉐론,티파니,219
13,반클리프,쎄뻥,180
22,불가리,쎄뻥,147
28,불가리,콰트로,121
43,까르띠에,콰트로,97
55,까르띠에,쎄뻥,89
64,반클리프,콰트로,82


In [35]:
edge_df['Type'] = 'Undirected'  #방향성이 없는 그래프이므로
edge_df.to_csv('edge_data_bch.csv', index=False) #index 는 0,1,2,3 등 각 row에 붙는 숫자인데 제외함.

In [36]:
edge_df

,Source,Target,Weight,Type
0,부쉐론,불가리,319,Undirected
1,목걸이,부쉐론,301,Undirected
2,다이아,부쉐론,285,Undirected
3,반클리프,부쉐론,284,Undirected
4,반지,부쉐론,279,Undirected
...,...,...,...,...
66,가방,부쉐론,80,Undirected
67,밴드,부쉐론,80,Undirected
68,디바스드림,부쉐론,78,Undirected
69,부쉐론,줄,77,Undirected


##### 3-2. node table: edge_df에 나온 단어들의 unique한 set

In [37]:
token = [t for tokens in df['nouns_document'] for t in list(set(tokens)) if not t in stop_words]

In [38]:
freq = nltk.FreqDist(token) #불가리 명사만 추출한 문서에서 빈도 분포

In [39]:
label_list = list(set(edge_df.Source)) + list(set(edge_df.Target))  #각 노드는 edge table에 등장한 유니크한 단어

In [40]:
#size_list 에는 각 단어가 원래 문서에서 몇 번 등장했는지
size_list = []
for label in label_list:
    size_list.append(freq[label])  

In [41]:
label = list(set(label_list)) 
len(label) #node개수

47

In [27]:
label_list = list(set(edge_df.Source)) + list(set(edge_df.Target))  #각 노드는 edge table에 등장한 유니크한 단어

In [28]:
#size_list 에는 각 단어가 원래 문서에서 몇 번 등장했는지
size_list = []
for label in label_list:
    size_list.append(freq[label])  

In [29]:
label = list(set(label_list))

In [42]:
node_df = pd.DataFrame(columns=['Label','Size'])

node_df['Id'] = label_list
node_df['Label'] = label_list
node_df['Size'] = size_list

In [ ]:
node_df

,Label,Size


In [44]:
node_df.to_csv('node_data_bch.csv', index =False)

In [ ]:
freq = nltk.FreqDist(token) #불가리 명사만 추출한 문서에서 빈도 분포(문서내의 단어 중복값 제거)

In [42]:
label_list = list(set(edge_df.Source)) + list(set(edge_df.Target))  #각 노드는 edge table에 등장한 유니크한 단어

In [43]:
label = list(set(label_list)) 
len(label) #node개수

70

In [44]:
#size_list 에는 각 단어가 원래 문서에서 몇 번 등장했는지
size_list = []
for label in label_list:
    size_list.append(freq[label])  

In [45]:
node_df = pd.DataFrame(columns=['Label','Size'])

node_df['Id'] = label_list
node_df['Label'] = label_list
node_df['Size'] = size_list

In [46]:
node_df.to_csv('node_data_bctv.csv', index =False)